In [ ]:
# Install necessary dependencies
!pip install huggingface_hub transformers datasets peft accelerate wandb


In [ ]:

import os
import re
from huggingface_hub import login
from transformers import AutoModelForTokenClassification, AutoTokenizer, Trainer, TrainingArguments
from datasets import Dataset
import torch
import wandb
from google.colab import drive
from peft import LoraConfig, get_peft_model

from google.colab import userdata
HUGGING_API = userdata.get('HUGGINGFACE_READ_AND_WRITE')
GOOGLE_API = userdata.get('GOOGLE_API_KEY')
WANDB_key = userdata.get('WANDB')


In [ ]:

# Login to Hugging Face
login(token=HUGGINGFACE_API)


In [ ]:

# Login to Weights & Biases
wandb.login(key=WANDB_key)


In [ ]:

run = wandb.init(project='llama-7b-hallucination', job_type="training", anonymous="allow", name="test_1")


In [ ]:

# Mount Google Drive to save models
drive.mount('/content/drive/')

In [ ]:

# Step 5: Create and Prepare Dataset for Hallucination Detection
# This dataset will have two fields: 'text' and 'labels'


In [ ]:

# 1. Load your dataset
data = [
    {"text": "The Eiffel Tower is located in Berlin, Germany.", "labels": [0, 0, 0, 0, 0, 1, 1]},  # Hallucinated words: "Berlin", "Germany"
    {"text": "The capital of France is Paris.", "labels": [0, 0, 0, 0]},  # Correct sentence
    {"text": "The Amazon River flows through Asia.", "labels": [0, 0, 0, 1]},  # Hallucinated word: "Asia"
]


In [ ]:

# 2. Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")


In [ ]:

# 3. Preprocessing function: Tokenizing by words and punctuation (treat punctuation as words)
def preprocess_function(examples):
    tokenized_input = []
    all_labels = []

    for idx, sentence in enumerate(examples['text']):
        word_labels = examples['labels'][idx]  # Get the corresponding labels for the sentence

        # Manually separate punctuation marks as individual "words"
        sentence = re.sub(r"([.,!?;])", r" \1 ", sentence)  # Add spaces around punctuation marks

        # Tokenize the sentence (this will now include punctuation as separate tokens)
        word_tokens = tokenizer.tokenize(sentence)

        # Split the sentence into words and punctuation for easier labeling
        words = sentence.split()

        # Ensure labels match the number of words
        assert len(word_labels) == len(words), f"Number of labels does not match the number of words in the sentence: {sentence}"

        # Create labels based on word-level tokens
        expanded_labels = []
        for word, label in zip(words, word_labels):
            word_token_count = len(tokenizer.tokenize(word))
            expanded_labels.extend([label] * word_token_count)  # Assign the label to each token of the word

        # Tokenize the entire sentence to get input_ids and attention_mask
        tokenized_sentence = tokenizer(sentence, padding="max_length", truncation=True, max_length=128)

        # Append the tokenized input and its corresponding labels
        tokenized_input.append(tokenized_sentence)
        all_labels.append(expanded_labels)

    return {
        'input_ids': [item['input_ids'] for item in tokenized_input],
        'attention_mask': [item['attention_mask'] for item in tokenized_input],
        'labels': all_labels
    }


In [ ]:

# 4. Convert data to Hugging Face dataset format
dataset = Dataset.from_dict({
    'text': [item['text'] for item in data],
    'labels': [item['labels'] for item in data]
})


In [ ]:

# 5. Apply the preprocessing
tokenized_data = dataset.map(preprocess_function, batched=True)


In [ ]:

# 6. Check tokenized data
print("Tokenized data:")
for item in tokenized_data:
    print(item)


In [ ]:

# Load model for token classification
model = AutoModelForTokenClassification.from_pretrained("meta-llama/Llama-2-7b-hf", num_labels=2)


In [ ]:

# Apply PEFT (LoRA)
lora_config = LoraConfig(
    r=8,  # Low-rank factor (adjust this based on your needs)
    lora_alpha=16,  # Scaling factor for LoRA
    lora_dropout=0.1,  # Dropout rate for LoRA
    bias="none"  # Specify whether or not to use biases in LoRA layers
)


In [ ]:

# Apply LoRA to the model
model = get_peft_model(model, lora_config)


In [ ]:

# Define label names (0 = correct, 1 = hallucinated)
model.config.id2label = {0: "correct", 1: "hallucinated"}
model.config.label2id = {"correct": 0, "hallucinated": 1}


In [ ]:

# Set up training arguments
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/NLP/MODELS/FineTunedModel",  # Path to save the model
    evaluation_strategy="epoch",  # Evaluate the model every epoch
    learning_rate=2e-5,  # Learning rate
    per_device_train_batch_size=4,  # Training batch size
    per_device_eval_batch_size=8,  # Evaluation batch size
    num_train_epochs=3,  # Number of epochs
    weight_decay=0.01,  # Weight decay for optimization
    logging_dir="/content/drive/MyDrive/NLP/MODELS/Logs",  # Save logs to Google Drive
    logging_steps=10,  # Log every 10 steps
    push_to_hub=False,  # Set to True to upload the model after training
    report_to="wandb",  # Report metrics to Weights & Biases
    run_name="test_1",
    resume_from_checkpoint=True,  # Automatically resume from the last checkpoint
    save_steps=300,
    save_total_limit=3, # keep only the last 3 checkpoints
)


In [ ]:

# Initialize the Trainer with model, training arguments, and datasets
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data,  # Use the tokenized dataset for training
    eval_dataset=tokenized_data,  # Optional: Use the same dataset for evaluation
    tokenizer=tokenizer,
)


In [ ]:

# Start training
trainer.train()


In [ ]:

# Save the fine-tuned model to Google Drive
model_save_path = "/content/drive/MyDrive/NLP/MODELS/FineTunedModel"
trainer.save_model(model_save_path)


In [ ]:

# Optionally, push the fine-tuned model to Hugging Face Hub
model.push_to_hub("your_huggingface_username/your_model_repo_name")
tokenizer.push_to_hub("your_huggingface_username/your_model_repo_name")


In [ ]:

wandb.finish()


In [ ]:

# Inference: Using the Fine-Tuned Model for Inference
def infer_with_model(input_text):
    # Tokenize the input text
    inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True, max_length=128)

    # Predict the token labels (hallucination vs. correct)
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits  # Raw logits output from the model

    # Get the predicted labels (0 for correct, 1 for hallucinated)
    predicted_labels = torch.argmax(logits, dim=-1)

    # Decode the tokens from the input text
    tokens = tokenizer.tokenize(input_text)

    # Get the corresponding predicted labels for each token
    labeled_tokens = list(zip(tokens, predicted_labels[0].tolist()))

    # Create a list of hallucinated words
    hallucinated_words = [token for token, label in labeled_tokens if label == 1]

    return hallucinated_words


In [ ]:

# Example usage of the inference function
input_text = "The Eiffel Tower is located in Berlin, Germany."
hallucinated_words = infer_with_model(input_text)

# Print the list of hallucinated words
print("Hallucinated words:")
print(hallucinated_words)
